In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import kruskal, mannwhitneyu
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def compute_lefse_biomarkers(abundance_df, group_series, p_threshold=0.05, lda_threshold=2.0):
    """
    Simulate the three-step process of LEfSe and output the biomarker table.
    args:
        abundance_df: pd.DataFrame，Behavioral characteristics (species), listed as samples, with values representing abundance.
        group_series: pd.Series，Sample grouping information (aligned with index and abundance_df.columns)
        p_threshold: float，Significance threshold of Kruskal and Wilcoxon
        lda_threshold: float，Minimum LDA score
    returns:
        pd.DataFrame，LEfSe result table
    """
    # Ensure sample alignment
    abundance_df = abundance_df.loc[:,
                                    abundance_df.columns.isin(group_series.index)]
    group_series = group_series.loc[abundance_df.columns]

    # Step 1: Kruskal-Wallis test
    pvals = {}
    for feature in abundance_df.index:
        groups = [abundance_df.loc[feature, group_series == g]
                  for g in group_series.unique()]
        if all(len(g) >= 2 for g in groups):
            stat, p = kruskal(*groups)
            pvals[feature] = p
        else:
            pvals[feature] = 1.0
    kw_df = pd.DataFrame.from_dict(pvals, orient='index', columns=['P_value'])
    kw_df = kw_df[kw_df['P_value'] < p_threshold]

    if kw_df.empty:
        print("⚠️ Features that did not pass the Kruskal-Wallis test")
        return pd.DataFrame()

    # Step 2: Wilcoxon test
    selected = []
    for feature in kw_df.index:
        values = abundance_df.loc[feature]
        sig = False
        unique_groups = group_series.unique()
        for i in range(len(unique_groups)):
            for j in range(i+1, len(unique_groups)):
                g1 = values[group_series == unique_groups[i]]
                g2 = values[group_series == unique_groups[j]]
                if len(g1) >= 2 and len(g2) >= 2:
                    _, p = mannwhitneyu(g1, g2, alternative='two-sided')
                    if p < p_threshold:
                        sig = True
                        break
            if sig:
                break
        if sig:
            selected.append(feature)

    if not selected:
        print("⚠️ Features that did not pass the Wilcoxon test")
        return pd.DataFrame()

    # Step 3: LDA analysis
    X = abundance_df.loc[selected].T.fillna(0).values
    y = group_series.values
    lda = LinearDiscriminantAnalysis()
    lda.fit(X, y)
    lda_scores = lda.coef_[0]

    features_selected = abundance_df.loc[selected].index.tolist()

    # abundance count
    max_abundance_log = []
    max_group = []
    for feature in features_selected:
        group_means = abundance_df.loc[feature].groupby(group_series).mean()
        max_group.append(group_means.idxmax())
        max_abundance_log.append(np.log10(group_means.max() + 1e-9))

    # result table
    result = pd.DataFrame({
        'Taxonomy': features_selected,
        'Log(max(meanAbundance))': max_abundance_log,
        'group': max_group,
        'LDA score': lda_scores,
        'P_value': kw_df.loc[features_selected, 'P_value'].values
    })

    # filter by LDA score
    result = result[abs(result['LDA score']) >= lda_threshold]
    result = result.sort_values(
        by='LDA score', ascending=False).reset_index(drop=True)

    return result

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr


def plot_microbiome_host_association_export_data(
    accession,
    taxonomy,
    work_dir,
    meta_file,
    host,
    expression_file,
    group_info,
    group_filter=None,
    p_threshold=0.05,
    lda_threshold=2.0,
    cor_p_filter=0.05
):
    """
    Calculate the correlation of microbial and host gene expression (after LEfSe selection), and create a significance mask for gene/species selection.

    returns:
        DataFrame: id, species, gene, correlation, accession
    """

    # 1. Obtain sample grouping
    pie_meta = pd.read_csv(meta_file, dtype=str)
    sub = pie_meta[(pie_meta['accession'] == accession)
                   & (pie_meta['info'] == group_info)]
    if group_filter:
        for key, val in group_filter.items():
            sub = sub[sub[key].isin(val)]

    if sub.empty:
        print(f"⚠️ {accession}-{group_info} No qualifying samples.")
        return None

    sample_list = sub['sample'].tolist()
    sample_to_meta = dict(zip(sub['sample'], sub['value']))

    # 2. Read the microbial abundance table.
    summary_path = os.path.join(work_dir, host, f"{accession}_summary.xlsx")
    if not os.path.exists(summary_path):
        print(f"❌ file not found: {summary_path}")
        return None

    abundance = pd.read_excel(summary_path, sheet_name=taxonomy, index_col=0).T
    abundance = abundance.loc[abundance.index.intersection(sample_list)]
    if abundance.shape[0] < 2:
        print(f"⚠️ Insufficient sample size, skipping {accession}")
        return None

    group_series = pd.Series(sample_to_meta)

    # 3. LEfSe 分析
    lefse_result = compute_lefse_biomarkers(
        abundance_df=abundance.T,
        group_series=group_series,
        p_threshold=p_threshold,
        lda_threshold=lda_threshold
    )
    if lefse_result.empty:
        print(f"⚠️ LEfSe no significant species, skipping {accession}")
        return None

    selected_species = lefse_result['Taxonomy'].tolist()

    # 4. Read gene expression data
    gene_exp = pd.read_csv(expression_file, sep="\t", index_col=0)
    gene_exp = gene_exp.loc[:, gene_exp.columns.intersection(sample_list)]

    # 5. Normalization
    micro_abu = abundance[selected_species].T
    micro_abu_norm = micro_abu.div(micro_abu.sum(axis=0), axis=1).fillna(0)
    gene_exp_norm = gene_exp.div(gene_exp.sum(axis=0), axis=1).fillna(0)

    # 6. Pearson correlation significance mask
    cor_result = []
    for sp in micro_abu_norm.index:
        for gene in gene_exp_norm.index:
            corr, p = pearsonr(micro_abu_norm.loc[sp], gene_exp_norm.loc[gene])
            cor_result.append([sp.split('|')[-1], gene, corr if p < cor_p_filter else 0, p])

    df_cor = pd.DataFrame(cor_result, columns=[
                          'species', 'gene', 'correlation', 'p_value'])
    df_cor['accession'] = accession

    # 7. Retain genes and microorganisms with non-zero relevance
    pivot = df_cor.pivot(index='species', columns='gene',
                         values='correlation').fillna(0)
    species_keep = pivot[(pivot != 0).any(axis=1)].index
    gene_keep = pivot.loc[species_keep].T[(
        pivot.loc[species_keep].T != 0).any(axis=1)].index

    df_final = df_cor[df_cor['species'].isin(
        species_keep) & df_cor['gene'].isin(gene_keep)]
    df_final = df_final[['species', 'gene', 'correlation', 'accession']]
    df_final.insert(0, 'id', range(len(df_final)))

    # 8. Build LDA table
    df_LDA = lefse_result[['Taxonomy', 'LDA score']].copy()
    df_LDA['Taxonomy'] = df_LDA['Taxonomy'].apply(lambda tax: tax.split('|')[-1])

    df_LDA.columns = ['species', 'LDA']
    df_LDA['accession'] = accession
    df_LDA.insert(0, 'id', range(len(df_LDA)))

    return df_final, df_LDA

In [ ]:
work_dir = "D:/Project/gutDBase/bracken_summary_GSE_filtered"
host = "human"
meta_file = "D:/Project/gutDBase/metadata/hum_pie.csv"

pie_meta = pd.read_csv(meta_file, dtype=str)
accessions = pie_meta['accession'].unique()
combo_table = pd.DataFrame({'accession': accessions, 'info': [
                            'metaclass'] * len(accessions)})

In [ ]:
all_data_association = []
all_data_LDA = []
taxonomy = 'species'
output_dir = os.path.join(
    "D:/Project/gutDBase/Microbiome_host_association_metaclass_csv", host)
exp_foler = os.path.join(
    "D:/Project/gutDBase/exp", host)
os.makedirs(output_dir, exist_ok=True)

for _, row in combo_table.iterrows():
    acc = row['accession']
    info = row['info']
    print(f"processing {acc} - {info}")
    try:
        association, LDA_score = plot_microbiome_host_association_export_data(
            accession=acc,
            group_info=info,
            taxonomy=taxonomy,
            work_dir=work_dir,
            meta_file=meta_file,
            host=host,
            expression_file=os.path.join(
                exp_foler, f"{acc}_bulk_GeneExpression.txt"))
        if association is not None:
            all_data_association.append(association)
        if LDA_score is not None:
            all_data_LDA.append(LDA_score)
    except Exception as e:
        print(f"⚠️ exception occurred: {e}")

if all_data_association:
    all_combined = pd.concat(all_data_association, ignore_index=True)
    out_path = os.path.join(
        output_dir, "combined_microbiome_host_association_data.csv")
    all_combined.to_csv(out_path, index=False)
    print(f"✅ All data has been successfully saved to {out_path}")
else:
    print("⚠️ No available data to export.")

if all_data_LDA:
    all_LDA = pd.concat(all_data_LDA, ignore_index=True)
    out_path = os.path.join(
        output_dir, "combined_LDA_score_data.csv")
    all_LDA.to_csv(out_path, index=False)
    print(f"✅ All data has been successfully saved to {out_path}")
else:
    print("⚠️ No available data to export.")